# Hello World Agent with UiPath SDK

# Before you begin

Make sure you created the `.venv` and installed `jupyter`.

In [ ]:
!py -m venv .venv
!.venv\Scripts\pip install ipykernel jupyter
!.venv\Scripts\python -m ipykernel install --user --name=jupyter-uv-env --display-name "WellsFargoLab"


### Select the kernel

![](./resources/prerequisites/3050-select-kernel.png)

![](./resources/prerequisites/3060-select-kernel.png)

![](./resources/prerequisites/3070-select-kernel.png)


In [ ]:
# If this doesn't work, nothing works!
!echo "You are all set, continue"

### Login to UiPath and navigate to Orchestrator

Navigate to [cloud.uipath.com](https://cloud.uipath.com/tpenlabs/AgenticILT) and login to the organization `tpenlabs`.

From the top right dropdown select the `AgenticILT` tenant.

The landing page will show you the available services. Go to Orchestrator.

![](./resources/prerequisites/0010-home-landing-page.png)

The Orchestrator service is where you configure and run your automations.

![](./resources/prerequisites/0011-orchestrator.png)


You have access to the WellsFargoCodedAgents folder. 

Create a subfolder using an unique name, example: `<your-name>`.

You will publish the packages to the tenant feed and define your automations in this folder.

![](./resources/prerequisites/2010-create-subfolder-2.png)

![](./resources/prerequisites/2010-create-subfolder-3.png)


### Add the uipath-langchain package

Your virtual environment was aleady setup and the Jupyter kernel was selected. 

Now it's time to add the [uipath-langchain](https://github.com/UiPath/uipath-langchain-python) package to the project. 

This package uses the [uipath](https://github.com/UiPath/uipath-python) package thaat provides APIs to access the UiPath platform, and adds helpers for developing and running agents.

This package is both the runtime dependency and the CLI utility.

In [ ]:
%pip install uipath-langchain

### Use the uipath cli to create a new agent

Use the `uipath new` to create a new hello world agent. 

Replace the placeholder with your unique name.

This command creates a new `main.py` file that contains the starting agent code that uses an `UiPathChat` to generate a report.


In [ ]:
!uipath new agent-<<< ADD YOUR NAME HERE >>>

### Authenticate to the UiPath Cloud

This will open the interactiv login page.

The login information will be stored in the `.env` file: organization key, tenant key, base URL, access token.

The base `uipath` sdk provides the helpers that will use the environmenet variables when making requests to the UiPath platform.

In [ ]:
!uipath auth --tenant AgenticILT

# Run the agent

You can provide agent input as a text or as a file.

In [ ]:
%%writefile input.json
{
    "topic": "AI Revolution 2025-2026"
}

You can run the agent locally using `uipath run`.

If the predefined model (`gpt-4o-mini-2024-07-18`) is rate-limited or not available you can use another UiPath-hosted model, example: `gpt-4.1-mini-2025-04-14`, `anthropic.claude-3-7-sonnet-20250219-v1:0`. Just edit the model name in the instantiation of `UiPathChat` in `main.py`.

In [ ]:
!uipath run agent --file input.json

### Pack an publish the agent

Before first packing and publishing the agent, you need to perform the initializaiton.

This creates the uipath.json file with the metadata needed for UiPath runtime to execute the agent.

It also creates the `agent.mermaid` diagram where you can see your graph diagram.

In [ ]:
!uipath init

Now use the uipath cli to package your code in a nuget package that is available under the `.uipath` folder.

The name and version from the `pyproject.toml` are used to build the pakcage name:

```
name = "agent-<your-name>"
version = "0.0.1"
```

The agent nuget package will be published in the Orchestrator tenant package feed.

The command will fail if another package with the same name and version exists. If this happens you must increment the version and run `uipath pack`.before publishing again.

In [ ]:
# Auto-increase the version of the package
import re, pathlib
p = pathlib.Path("pyproject.toml"); t = p.read_text()
n = re.sub(r'(?m)^(version\s*=\s*")(\d+)\.(\d+)\.(\d+)(")', lambda m: f'{m.group(1)}{m.group(2)}.{m.group(3)}.{int(m.group(4))+1}{m.group(5)}', t, count=1)
p.write_text(n); print(re.search(r'(?m)^version\s*=\s*"([^"]+)"', t).group(1), "->", re.search(r'(?m)^version\s*=\s*"([^"]+)"', n).group(1))

In [ ]:
!uipath pack
!uipath publish -f WellsFargoCodedAgents

### Create a process in the folder

Use your uploaded package to create a `Process` in your folder.

The `Process` allows additional configuration that are specific to this folder, such as predefining `Environment Configuration` or `Input Args`.

First select the Package for this Process. Make sure you select the correct agent, keep in mind that you are selecting from the Tenant feed (packages for all folders are stored there).

![](./resources/prerequisites/0060-create-process.png)

![](./resources/prerequisites/0061-create-process.png)

![](./resources/prerequisites/0062-create-process.png)

![](./resources/prerequisites/0063-create-process.png)


### Start the process 

On the `Processes` list, use the `Start a Job` button next to the three-dots menu.

The Job will be executed using a Robot identity and a Serverless machine.

![](./resources/prerequisites/0070-start-job.png)

Supply the input arguments for the job.

![](./resources/prerequisites/0071-start-job.png)

View the job details using the info button on the grid.

![](./resources/prerequisites/0073-start-job.png)

The job completed successfully.

![](./resources/prerequisites/0074-start-job.png)